In [1]:
from io import StringIO
import numpy as np

# Загрузка данных из файла
 и пример того, как много разных параметров мы можем задать при чтении данных из файла - потому важно читать документацию :)
 Обычно то, что мы можем задать ограничивается стандартны набором параметров.

## [np.loadtxt](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) и [np.genfromtxt](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html)

Обе фукнции предназначены для считывания данных из файла. 
Функция loadtxt предназначена для быстрого чтения просто отформатированных файлов. Функция genfromtxt обеспечивает более гибкую обработку, например, строк с пропущенными значениями.
genfromtxt имеет все функции, которые и loadtxt + дополнительные. Далее в подразделах с демонстрацией функций, содержащихся в обеих библиотеках, я буду использовать loadtxt, а там, где функционал особенный для genfromtxt - эту библиотеку.

Простой способ считать данные из файла простой структуры.

In [2]:
data = u"1, 2, 3\n4, 5, 6"
np.loadtxt(StringIO(data), delimiter=",")

array([[1., 2., 3.],
       [4., 5., 6.]])

Единственный обязательный аргумент loadtxt - это источник данных. Это может быть строка, список строк, генератор или открытый файловый объект с методом чтения, например файл или объект io.StringIO. Если предоставляется одна строка, предполагается, что это имя локального или удаленного (который не локально, а где-то скажем в облаке) файла. Если предоставляется список строк или генератор, возвращающий строки, каждая строка рассматривается как одна строка в файле. Когда передается URL-адрес удаленного файла, файл автоматически загружается в текущий каталог и открывается.

Распознаваемые типы файлов - это текстовые файлы и архивы. В настоящее время функция распознает архивы gzip и bz2 (bzip2). Тип архива определяется расширением файла: если имя файла заканчивается на '.gz', ожидается архив gzip; если он заканчивается на "bz2", предполагается архив bzip2.

Разделителем может быть некоторый символ

In [3]:
!cat data.txt

1,13,21,11,196,75,4,3,34,6,7,8,0,1,2,3,4,5
3,42,12,33,766,75,4,55,6,4,3,4,5,6,7,0,11,12
1,22,33,11,999,11,2,1,78,0,1,2,9,8,7,1,76,88

In [4]:
filedata = np.loadtxt('data.txt', delimiter=',')
filedata = filedata.astype('int32')
print(filedata)

[[  1  13  21  11 196  75   4   3  34   6   7   8   0   1   2   3   4   5]
 [  3  42  12  33 766  75   4  55   6   4   3   4   5   6   7   0  11  12]
 [  1  22  33  11 999  11   2   1  78   0   1   2   9   8   7   1  76  88]]


Вот так считать не получится:

In [5]:
np.loadtxt('data.txt', delimiter=';')

ValueError: could not convert string to float: '1,13,21,11,196,75,4,3,34,6,7,8,0,1,2,3,4,5'

Файл может иметь фиксированную длину, тогда разделитель - кол-во символов. В этом случае нам нужно установить delimiter на одно целое число (если все столбцы имеют одинаковый размер) или последовательность целых чисел (если столбцы могут иметь разные размеры). Это возможно только в genfromtxt.

In [6]:
data = u"  1  2  3\n\n\n  4  5 67\n890123  4"
np.genfromtxt(StringIO(data), delimiter=3)

array([[  1.,   2.,   3.],
       [  4.,   5.,  67.],
       [890., 123.,   4.]])

In [7]:
data = u"123456789\n   4  7 9\n   4567 9"
np.genfromtxt(StringIO(data), delimiter=(4, 3, 2))

array([[1234.,  567.,   89.],
       [   4.,    7.,    9.],
       [   4.,  567.,    9.]])

In [8]:
print(data)

123456789
   4  7 9
   4567 9


## Аргумент autostrip

Доступно только в genfromtxt.  
По умолчанию, когда при чтении строка разбивается на серию строк, ведущий и замыкающие пробелы не удаляются.   
Это поведение можно изменить, установив для необязательного аргумента autostrip значение True.

In [9]:
'   abc  '

'   abc  '

In [10]:
data = u"1, фив , 2\n 3, xxx, 4"
# Without autostrip
np.genfromtxt(StringIO(data), delimiter=",", dtype="|U5")

array([['1', ' фив ', ' 2'],
       ['3', ' xxx', ' 4']], dtype='<U5')

In [11]:
# With autostrip
np.genfromtxt(StringIO(data), delimiter=",", dtype="|U5", autostrip=True)

array([['1', 'фив', '2'],
       ['3', 'xxx', '4']], dtype='<U5')

## Пропуск строк и выбор столбцов: skip_header / skip_footer

Доступно только в genfromtxt.  
Наличие заголовка в файле может затруднить обработку данных.   
В этом случае нам нужно использовать необязательный аргумент skip_header.  
Значения этого аргумента должны быть целым числом, которое соответствует количеству строк, которые нужно пропустить в начале файла перед выполнением любого другого действия.   
Точно так же мы можем пропустить последние n строк файла, используя атрибут skip_footer.
По умолчанию значения обоих аргументов = 0.

In [12]:
data = u"\n".join(str(i) for i in range(10))

In [13]:
print(data)

0
1
2
3
4
5
6
7
8
9


In [14]:
np.genfromtxt(StringIO(data),)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [15]:
np.genfromtxt(StringIO(data), skip_header=3, skip_footer=5)

array([3., 4.])

## Выбор количества строк для чтения - max_rows

`skiprows` - аналог `skip_header`, но в `np.loadtxt`  
`max_rows` - сколько строк мы хотим считать после того, как пропустили сколько-то первых строк.

Без max_rows:

In [16]:
np.loadtxt(StringIO(data), skiprows=3)

array([3., 4., 5., 6., 7., 8., 9.])

С max_rows:

In [17]:
np.loadtxt(StringIO(data), skiprows=3, max_rows=4)

array([3., 4., 5., 6.])

## usecols

В некоторых случаях нас интересуют не все столбцы данных, а только некоторые из них. Мы можем выбрать, какие столбцы импортировать, с помощью аргумента usecols. Этот аргумент принимает одно целое число или последовательность целых чисел, соответствующих индексам импортируемых столбцов. Помните, что по соглашению первый столбец имеет индекс 0. Отрицательные целые числа ведут себя так же, как обычные отрицательные индексы Python.

Например, если мы хотим импортировать только первый и последний столбцы, мы можем использовать usecols = (0, -1).

In [18]:
print(data)

0
1
2
3
4
5
6
7
8
9


In [19]:
data = u"1 2 3\n4 5 6"
np.genfromtxt(StringIO(data), usecols=(0, -1))

array([[1., 3.],
       [4., 6.]])

In [20]:
print(data)

1 2 3
4 5 6


In [21]:
data = u"1 2 3\n4 5 6"
np.genfromtxt(StringIO(data), names="a, b, c", usecols=("a", "c"))

array([(1., 3.), (4., 6.)], dtype=[('a', '<f8'), ('c', '<f8')])

## Выбор типа данных 

Основной способ контролировать, как последовательности строк, которые мы прочитали из файла, преобразуются в другие типы, - это установить аргумент dtype. Допустимые значения для этого аргумента:

- единственный тип, например dtype = float. Вывод будет двухмерным с заданным dtype, если имя не было связано с каждым столбцом с использованием аргумента имен. По умолчанию для genfromtxt используется dtype = float.

- последовательность типов, например dtype = (int, float, float).

- строка, разделенная запятыми, например dtype = "i4, f8, | U3".

- словарь с двумя ключами "names" и "formats".

- последовательность кортежей (имя, тип), например dtype = [('A', int), ('B', float)].

- существующий объект numpy.dtype.

- специальное значение  None. В этом случае тип столбцов будет определяться из самих данных.

Во всех случаях, кроме первого, на выходе будет одномерный массив со структурированным dtype. Этот dtype имеет столько же полей, сколько элементов в последовательности. Имена полей определяются с помощью ключевого слова names.

Когда dtype = None, тип каждого столбца определяется итеративно по его данным.   
Мы начинаем с проверки, может ли строка быть преобразована в логическое значение (то есть соответствует ли строка истинному или ложному в нижнем регистре);  
затем можно ли его преобразовать в целое число, затем в число с плавающей запятой, затем в комплексное и, наконец, в строку.  
Это поведение можно изменить, изменив сопоставитель по умолчанию класса StringConverter.

Опция dtype = None предоставляется для удобства. Однако это значительно медленнее, чем явная установка dtype.

In [22]:
data = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(data, dtype=(int, float, int))

array([(1, 2., 3), (4, 5., 6)],
      dtype=[('f0', '<i8'), ('f1', '<f8'), ('f2', '<i8')])

### Установка имен: names
Естественный подход при работе с табличными данными - присвоить имя каждому столбцу. Первая возможность - использовать явный структурированный dtype, как упоминалось ранее.

In [23]:
[(_, int) for _ in "abc"]

[('a', int), ('b', int), ('c', int)]

In [24]:
data = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(data, dtype=[(_, int) for _ in "abc"])

array([(1, 2, 3), (4, 5, 6)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

Другая более простая возможность - использовать ключевое слово names с последовательностью строк или строкой, разделенной запятыми:

In [25]:
data = StringIO("1 2 3\n 4 5 6")
np.genfromtxt(data, names="A, B, C")

array([(1., 2., 3.), (4., 5., 6.)],
      dtype=[('A', '<f8'), ('B', '<f8'), ('C', '<f8')])

В приведенном выше примере мы использовали тот факт, что по умолчанию dtype = float. Задавая последовательность имен, мы форсируем вывод в структурированный dtype.

Иногда нам может потребоваться определить имена столбцов из самих данных. В этом случае мы должны использовать ключевое слово names со значением True. Имена будут считываться из первой строки (после skip_header), даже если строка закомментирована:

In [26]:
data = "So it goes\n#a b c\n1 2 3\n 4 5 6"
np.genfromtxt(StringIO(data), skip_header=1, names=True)

array([(1., 2., 3.), (4., 5., 6.)],
      dtype=[('a', '<f8'), ('b', '<f8'), ('c', '<f8')])

In [27]:
print(data)

So it goes
#a b c
1 2 3
 4 5 6


In [28]:
print("So it goes\n#a b c\n1 2 3\n 4 5 6")

So it goes
#a b c
1 2 3
 4 5 6



Значение по умолчанию для имен - None. Если мы зададим для этого ключевого слова любое другое значение, новые имена перезапишут имена полей, которые мы, возможно, определили с помощью dtype:

In [29]:
data = StringIO("1 2 3\n 4 5 6")
ndtype=[('a',int), ('b', float), ('c', int)]
names = ["A", "B", "C"]
np.genfromtxt(data, names=names, dtype=ndtype)

array([(1, 2., 3), (4, 5., 6)],
      dtype=[('A', '<i8'), ('B', '<f8'), ('C', '<i8')])

## Поиск пропущенных значений и их заполнение 
Это - основное преимущество np.genfromtxt над np.loadtxt.

Некоторые записи могут отсутствовать в наборе данных, который мы пытаемся импортировать.   
При этом пользовательские преобразователи (к примеру, с помощью конвертеров, про которые идёт речь в предыдущем пункте) могут быстро стать громоздкими в управлении.  

Функция genfromtxt предоставляет два других дополнительных механизма: 
- аргумент **missing_values**  
используется для распознавания отсутствующих данных


- аргумент **fill_values**  
используется для обработки этих недостающих данных.

### missing_values
По умолчанию любая пустая строка помечается как отсутствующая. Мы также можем рассмотреть более сложные строки, такие как «N/A» или «???» для того чтобы представить отсутствующие или некорректные данные. Аргумент missing_values принимаут следующие три типа данных:
- строка или строка, разделенная запятыми  
Эта строка будет использоваться в качестве маркера отсутствующих данных для всех столбцов.


- последовательность строк    
В этом случае каждый элемент по порядку связан со столбцом.


- словарь  
Значения словаря - это строки или последовательность строк. Соответствующие ключи могут быть индексами столбцов (целыми числами) или именами столбцов (строками).   
Кроме того, специальный ключ None можно использовать для определения значения по умолчанию, применимого ко всем столбцам.


### fill_values
Мы знаем, как распознать отсутствующие данные, но нам все равно нужно указать значение для этих отсутствующих записей. По умолчанию это значение определяется из ожидаемого dtype в соответствии с таблицей на слайде.
Мы можем получить более точный контроль над преобразованием отсутствующих значений с помощью необязательного аргумента fill_values. Как и missing_values, этот аргумент принимает значения разных типов:

- единственное значение  
Это будет по умолчанию для всех столбцов.


- последовательность значений  
Каждая запись будет по умолчанию для соответствующего столбца.


- словарь  
Каждый ключ может быть индексом столбца или именем столбца, а соответствующее значение должно быть одним объектом. Мы можем использовать специальный ключ None, чтобы определить значение по умолчанию для всех столбцов.


В следующем примере мы предполагаем, что отсутствующие значения помечены как «N/A» в первом столбце и «???» в третьем столбце.   
Мы хотим преобразовать эти отсутствующие значения в 0, если они встречаются в первом и втором столбце, и в -999, если они встречаются в последнем столбце:

In [30]:
data = u"N/A, 2, 3\n4, ,???"

In [31]:
print(data)

N/A, 2, 3
4, ,???


In [32]:
kwargs = dict(delimiter=",",
               dtype=int,
               names="a,b,c",
               missing_values={0:"N/A", 'b':" ", 2:"???"},
               filling_values={0:0, 'b':0, 2:-999})

np.genfromtxt(StringIO(data), **kwargs)

array([(0, 2,    3), (4, 0, -999)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

In [33]:
kwargs['dtype'] = float

In [34]:
kwargs

{'delimiter': ',',
 'dtype': float,
 'names': 'a,b,c',
 'missing_values': {0: 'N/A', 'b': ' ', 2: '???'},
 'filling_values': {0: 0, 'b': 0, 2: -999}}

In [35]:
np.genfromtxt(StringIO(data), kwargs)

ValueError: entry not a 2- or 3- tuple

In [36]:
list(zip([1,2,3], [4,5,6]))

[(1, 4), (2, 5), (3, 6)]

#  Запись в файл

## numpy.save

numpy.save(file, arr, allow_pickle=True, fix_imports=True)  
Сохраняет массив в файл формата .npy.

Параметры:  

- file: file, str или pathlib.Path
Файл или имя файла, в котором сохраняются данные. Если файл является файл-объектом, то имя файла не изменяется. Если файл представляет собой строку или путь, к имени файла будет добавлено расширение .npy, если его еще нет.


- arr: array_like  
Сохраняемые данные массива.


- allow_pickle: bool, необязательно  
Разрешить сохранение массивов объектов с помощью Python pickles ("консервы"."закрутки"). Причины запрета на использование pickles включают безопасность (загрузка pickled данных может выполнять произвольный код) и переносимость (pickled объекты могут не загружаться на разных версиях Python, например, если для сохраненных объектов требуются библиотеки, которые недоступны, и не все pickled данные совместимы между Python 2 и Python 3). По умолчанию: True


- fix_imports: bool, необязательно
Полезно только для того, чтобы заставить объекты в массивах объектов на Python 3 обрабатываться совместимым с Python 2 способом. Если fix_imports имеет значение True, pickle попытается сопоставить новые имена Python 3 со старыми именами модулей, используемыми в Python 2, чтобы поток данных pickle был доступен для чтения с Python 2.

**Заметки:**
    
Детально про формат .npy можно почитать в [документации](https://numpy.org/doc/stable/reference/generated/numpy.lib.format.html#module-numpy.lib.format).  

Любые данные, сохраненные в файле, добавляются в конец файла.

## Формат NPY
Простой формат для сохранения NumPy массивов на диск с полной информацией о них.

Формат **.npy** - это стандартный двоичный формат файла в NumPy для сохранения одного произвольного массива NumPy на диске.
Формат хранит всю информацию о форме и dtype, необходимую для правильного восстановления массива даже на другом компьютере с другой архитектурой.  
Формат разработан так, чтобы быть максимально простым, но при этом достигать своих ограниченных целей.

Формат **.npz** - это стандартный формат для хранения **нескольких** массивов NumPy на диске. 
Файл .npz - это zip-файл, содержащий несколько файлов .npy, по одному для каждого массива.

### Почему данные стоит хранить в npy вместо csv [полный ответ \[ENG\]](https://www.reddit.com/r/datascience/comments/blqa4v/why_you_should_always_save_your_data_as_npy/)

Обычно данные в .npy формате занимают меньше места на диске чем файлы в csv и txt. 

[Статья со сравнением](https://medium.com/nuances-of-programming/%D0%BF%D0%BE%D1%87%D0%B5%D0%BC%D1%83-%D0%B2%D1%8B-%D0%B4%D0%BE%D0%BB%D0%B6%D0%BD%D1%8B-%D0%BD%D0%B0%D1%87%D0%B0%D1%82%D1%8C-%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D1%8C-npy-%D1%84%D0%B0%D0%B9%D0%BB-%D1%87%D0%B0%D1%89%D0%B5-80989dd85095)

Но важно учитывать [нюансы \[ENG\]](https://stackoverflow.com/questions/53407468/why-is-the-size-of-npy-bigger-than-csv).

Пишем в файл мы очень просто.

In [37]:
data = StringIO("11 12 13\n 44 45 46")
np_data = np.genfromtxt(data, dtype=[(_, int) for _ in "abc"])

In [38]:
np_data

array([(11, 12, 13), (44, 45, 46)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

In [39]:
with open('test.npy', 'wb') as f:
    np.save(f, np_data)

In [40]:
np.save(open('test.npy', 'wb'), np_data)

In [41]:
! cat test.npy

�NUMPY v {'descr': [('a', '<i8'), ('b', '<i8'), ('c', '<i8')], 'fortran_order': False, 'shape': (2,), }                       
       ,       -       .       

In [42]:
np.load('test.npy')

array([(11, 12, 13), (44, 45, 46)],
      dtype=[('a', '<i8'), ('b', '<i8'), ('c', '<i8')])

# Линейная алгебра

## Работа с векторами

Сложение векторов

In [43]:
v = np.array([3, 7])
u = np.array([2, 2])

v + u

array([5, 9])

In [44]:
v, u

(array([3, 7]), array([2, 2]))

In [45]:
v = np.array([3, 7])

20 * v

array([ 60, 140])

In [46]:
v = np.array([3, 7])
u = np.array([2, 2])

v.dot(u)

20

In [47]:
np.dot(v,u)

20

In [48]:
v = np.array([3, 2, 7])

np.linalg.norm(v)

7.874007874011811

In [49]:
def angle_between(v1, v2):
    dot_pr = v1.dot(v2)
    norms = np.linalg.norm(v1) * np.linalg.norm(v2)
    return np.rad2deg(np.arccos(dot_pr / norms))

v = np.array([1, 4, 5])
u = np.array([2, 1, 5])

angle_between(v, u)

29.152519407030084

## Работа с матрицами

Сложение матриц

In [50]:
A = np.matrix([
    [3, 5],
    [1, 0]
])
B = np.matrix([
    [2, -3],
    [1, 2]
])

A + B

matrix([[5, 2],
        [2, 2]])

In [51]:
A, B

(matrix([[3, 5],
         [1, 0]]),
 matrix([[ 2, -3],
         [ 1,  2]]))

Скалярное умножение

In [52]:
A = np.matrix([
    [3, 5],
    [1, 0]
])

2 * A

matrix([[ 6, 10],
        [ 2,  0]])

Матричное умножение

In [53]:
A = np.matrix([
    [3, 4],
    [1, 0]
])
B = np.matrix([
    [2, 2],
    [1, 2]
])

A.dot(B)

matrix([[10, 14],
        [ 2,  2]])

Тот же результат при использовании звёздочки:

In [54]:
A * B

matrix([[10, 14],
        [ 2,  2]])

**НО** если используем тип array для матриц - умножение через звёздочку - поэлементное.

In [55]:
A = np.array([
    [3, 4],
    [1, 0]
])
B = np.array([
    [2, 2],
    [1, 2]
])

In [56]:
A * B

array([[6, 8],
       [1, 0]])

In [57]:
np.matmul(A,B)

array([[10, 14],
       [ 2,  2]])

In [58]:
A.dot(B)

array([[10, 14],
       [ 2,  2]])

Транспонированная матрица

In [59]:
A = np.matrix([
    [3, 4],
    [1, 10]
])

A.T

matrix([[ 3,  1],
        [ 4, 10]])

Альтернативно:

In [60]:
np.transpose(A)

matrix([[ 3,  1],
        [ 4, 10]])

Определитель матрицы

In [61]:
A = np.matrix([
    [3, 2],
    [1, 6]
])

np.linalg.det(A)

16.000000000000007

Обратная матрица

In [62]:
A  = np.matrix([
    [4, 3],
    [5, 4]
])

In [63]:
np.linalg.inv(A)

matrix([[ 4., -3.],
        [-5.,  4.]])

In [64]:
A.dot(np.linalg.inv(A))

matrix([[1., 0.],
        [0., 1.]])

In [65]:
np.linalg?

# Статистика

In [66]:
import math

In [67]:
x = [1, 2.5, 4, 8, 28]
x_with_nan = [8.0, 1, 2.5, math.nan, 4, 28.0]

In [68]:
x, x_with_nan

([1, 2.5, 4, 8, 28], [8.0, 1, 2.5, nan, 4, 28.0])

In [69]:
y, y_with_nan = np.array(x), np.array(x_with_nan)

In [70]:
y, y_with_nan 

(array([ 1. ,  2.5,  4. ,  8. , 28. ]),
 array([ 8. ,  1. ,  2.5,  nan,  4. , 28. ]))

Среднее

In [71]:
mean_ = sum(x) / len(x)
mean_

8.7

In [72]:
np.mean(x)

8.7

In [73]:
x.mean()

AttributeError: 'list' object has no attribute 'mean'

In [74]:
y.mean()

8.7

In [75]:
np.mean(x_with_nan)

nan

In [76]:
x

[1, 2.5, 4, 8, 28]

In [77]:
x_with_nan

[8.0, 1, 2.5, nan, 4, 28.0]

In [78]:
np.nanmean(x_with_nan)

8.7

Медиана

In [79]:
np.median(x)

4.0

In [80]:
np.median(y)

4.0

In [81]:
y.median()

AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [82]:
np.dot(y,x)

871.25

In [83]:
np.median(y_with_nan)

nan

In [84]:
np.nanmedian(y_with_nan)

4.0

Мода

In [85]:
u = [2, 3, 2, 8, 12, 2, 2]
mode = max((u.count(item), item) for item in set(u))[1]

In [86]:
mode

2

In [87]:
import statistics

In [88]:
statistics.mode(u)

2

In [89]:
statistics.multimode(u)

[2]

In [90]:
v = [12, 15, 12, 15, 21, 15, 12]

In [91]:
statistics.mode(v)

12

In [92]:
statistics.multimode(v)

[12, 15]

In [93]:
statistics.multimode([2, math.nan, 0, math.nan, 5, 2])

[2, nan]

Мода на многомерном массиве

In [94]:
A = [[1,3,4,2,2,7], [5,3,2,1,4,1], [3,3,2,2,1,1]]

In [95]:
A

[[1, 3, 4, 2, 2, 7], [5, 3, 2, 1, 4, 1], [3, 3, 2, 2, 1, 1]]

In [96]:
statistics.mode(A)

TypeError: unhashable type: 'list'

In [97]:
from scipy.stats import mode

In [98]:
mode(A)

ModeResult(mode=array([[1, 3, 2, 2, 1, 1]]), count=array([[1, 3, 2, 2, 1, 2]]))

Имплементация на numpy:

In [99]:
np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=A)

array([1, 3, 2, 2, 1, 1])

## Метрики оценки вариативности данных

Стандартное отклонение

ddof : int, optional
    Means Delta Degrees of Freedom.  The divisor used in calculations
    is ``N - ddof``, where ``N`` represents the number of elements.
    By default `ddof` is zero.

В случае выборки

In [100]:
np.std(y, ddof=1)

11.099549540409285

В случае совокупности (много данных)

In [101]:
np.std(y)

9.927738916792684

In [102]:
y

array([ 1. ,  2.5,  4. ,  8. , 28. ])

Внимание: в пакете statistics по умолчанию в функции stdev имплементирован вариант для выборки, а в Numpy - для совокупности.

In [103]:
statistics.stdev(y)

11.099549540409287

In [104]:
statistics.pstdev(y)

9.927738916792686

Персентили

In [105]:
x = [-5.0, -1.1, 0.1, 2.0, 8.0, 12.8, 21.0, 25.8, 41.0]

In [106]:
sorted(x)

[-5.0, -1.1, 0.1, 2.0, 8.0, 12.8, 21.0, 25.8, 41.0]

In [107]:
np.percentile(x, 5)

-3.44

In [108]:
np.percentile(x, 95)

34.919999999999995

In [109]:
np.percentile(x, 100)

41.0

In [110]:
np.percentile(x, [25, 50, 75])

array([ 0.1,  8. , 21. ])

Диапазон

In [111]:
y = np.array(x)

In [112]:
y.min(), y.max()

(-5.0, 41.0)

In [113]:
np.min(x), np.max(x)

(-5.0, 41.0)

В случае многомерного массива

In [114]:
a = np.array([[0, 1, 6],
              [21, 41, 11]])

In [115]:
a.min(axis=1)

array([ 0, 11])

In [116]:
a.min(axis=0)

array([0, 1, 6])

In [117]:
a.min()

0